In [1]:
import os
import json
import re
import tempfile
from pdf2image import convert_from_path
import img2pdf
import fitz
from PIL import Image
import io
import pandas as pd
import time
from langchain_upstage import UpstageLayoutAnalysisLoader
# from rainbow_html_transformer_old import HTMLToTextWithMarkdownTables
from rainbow_html_transformer import HTMLToTextWithIndentation
from tqdm import tqdm

In [2]:
def get_pdf_pages(pdf_path):
    doc = fitz.open(pdf_path)
    return len(doc)

In [3]:
def extract_text_with_ocr(pdf_path, max_pages=3):
    extracted_text = []
    html_transformer = HTMLToTextWithIndentation()
    
    try:
        images = convert_from_path(pdf_path, first_page=1, last_page=max_pages)
        
        print(f"Processing {len(images)} pages...")
        for i, img in enumerate(tqdm(images, desc="Processing pages", unit="page")):
            with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as temp_file:
                img.save(temp_file, format="PNG")
                temp_file_path = temp_file.name
            
            try:
                loader = UpstageLayoutAnalysisLoader(
                    file_path=temp_file_path,
                    use_ocr=True
                )
                documents = loader.load()
                
                if not documents:
                    print(f"Warning: No content extracted from page {i+1}")
                    continue

                page_content = ""
                for doc in documents:
                    if doc is None or not doc.page_content:
                        print(f"Warning: Empty document encountered for page {i+1}")
                        continue
                    transformed_doc = html_transformer.transform_document(doc)
                    page_content += transformed_doc.page_content

                # 페이지 내용 정리
                page_content = page_content.strip()
                if page_content:
                    page_content += f"\n\n--- End of Page {i+1} ---\n\n"
                    extracted_text.append(page_content)
                else:
                    print(f"Warning: No content extracted from page {i+1}")
            except Exception as e:
                print(f"Error processing page {i+1}: {e}")
            finally:
                os.unlink(temp_file_path)
    
    except Exception as e:
        print(f"Error processing {pdf_path} with OCR: {e}")
    
    if not extracted_text:
        print("Warning: No text was extracted from the document.")
    
    return ''.join(extracted_text)

In [4]:
# 사용 예:
# Layout Analysis API의 최대 페이지 제한인 100페이지 이상은 처리 불가
folder_path = "./raw_docs"
output_folder_path = "./processed_txt"
# pdf_file_name = "판매약관_ThePride신한참좋은치아보험PlusⅡ(무배당, 갱신형)_20240401_P252.pdf"
# pdf_file_name = "SHL0165_The안심VIP저축보험Ⅱ(무배당)_P116.pdf"
# pdf_file_name = "DB자산관리_약관_20240401_P22.pdf"
pdf_file_name = "IRP자산관리_약관(기업형)_20240401_P24.pdf"
print(f"Processing file : {pdf_file_name}")
pdf_path = os.path.join(folder_path, pdf_file_name)

max_pages = get_pdf_pages(pdf_path)
ocr_text = extract_text_with_ocr(pdf_path, max_pages)

# 텍스트 파일로 저장
output_file_name = os.path.splitext(pdf_file_name)[0] + ".txt"
output_path = os.path.join(output_folder_path, output_file_name)

with open(output_path, 'w', encoding='utf-8') as f:
    f.write(ocr_text)

print(f"OCR text saved to: {output_path}")

# 저장된 텍스트 출력 (선택사항)
print("Extracted text:")
print(ocr_text[:500] + "...")  # 처음 500자만 출력

Processing file : IRP자산관리_약관(기업형)_20240401_P24.pdf
Processing 24 pages...


Processing pages: 100%|██████████| 24/24 [01:23<00:00,  3.48s/page]

OCR text saved to: ./processed_txt/IRP자산관리_약관(기업형)_20240401_P24.txt
Extracted text:
무배당 신한개인형퇴직연금 자산관리보험
      약관
  (기업형)

--- End of Page 1 ---

무배당 신한개인형퇴직연금 자산관리보험 약관
        (기업형)
    제1조 (약관의 목적)
    이 약관의 목적은 근로자퇴직급여보장법(이 약관에서 "법"이라 합니다) 제25조에 의
    하여 개인형퇴직연금(기업형)제도(이 약관에서 "이 제도"라 합니다)를 설정한 가입자
    와 퇴직연금사업자인 신한라이프생명보험주식회사(이 약관에서 "회사"라 합니다)가
    이 제도의 자산관리업무 수행을 위하여 개인형퇴직연금(기업형) 자산관리(보험)계약
    (이 약관에서 "이 계약"이라 합니다)을 체결함에 있어 세부사항을 정함에 있습니다.
    제2조 (용어의 정의)
    ① 이 약관에서 사용하는 용어의 정의는 다음과 같습니다.
    1. "사용자"란「근로기준법」 제2조 제1항 제2호에 따른 사용자를 말합니다.
    2. "가입자"라 함은 법 제25조 제2항에 의해 ...
